In [1]:
import os 
import cv2 
import PIL 
import random
from glob import glob 
import pandas as pd 
import numpy as np 

---
PAMAP2 video action data version-1 Skeleton movement generations

In [2]:
import mediapipe as mp

In [3]:
# initialize Pose estimator
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [4]:
def pose_estimator(pose, path):
    # create capture object
    cap = cv2.VideoCapture(path)
    if (cap.isOpened() == False): 
        print("Error reading video file")
    
    # We need to set resolutions.
    # so, convert them from float to integer.
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    size = (frame_width, frame_height)

    skeleton_video = []
    while cap.isOpened():
        # read frame from capture object
        _, frame = cap.read()

        try:
            # convert the frame to RGB format
            RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # process the RGB frame to get the result
            results = pose.process(RGB)
            try:
                frame_result = results.pose_landmarks.landmark
                point_list = []
                for idx, landmark in enumerate(frame_result):
                    # print(idx, landmark.x)
                    # print(type(landmark))
                    coord = [landmark.x, landmark.y, landmark.z]
                    point_list.append(coord)
                skeleton_video.append(point_list)
            except Exception as inst:
                print(inst)
                pass
            
        except Exception as inst:
            print(inst)
            break
        if cv2.waitKey(1) == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

    return np.array(skeleton_video), frame_height, frame_width

In [5]:
active_points = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]
point_mapping = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]


In [6]:
def gen_skeleton(frame, connections, height, width):
    img_3 = np.zeros([height, width,3],dtype=np.uint8)
    img_3.fill(255)

    # add circles 
    for coord in frame:
        x, y = int(width*coord[0]), int(height*coord[1])
        img_3 = cv2.circle(img_3, center=(x,y), radius=1, color=(255, 0, 0), thickness=6)

    # add lines 
    # mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)]
    mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]
    for line in mapping_list:
        i, j = line
        st = frame[i, :]
        start_point = (int(width*st[0]), int(height*st[1]))

        en = frame[j, :]
        end_point = (int(width*en[0]), int(height*en[1]))

        img3_ = cv2.line(img_3, start_point, end_point, color=(0, 0, 0), thickness=3)

    return img_3

In [7]:
def gen_video(points, save_file, frame_h, frame_w):
    # make 3D if points are flatten 
    if len(points.shape) == 2:
        fts = points.shape[1]
        x_cds = list(range(0, fts, 3))
        y_cds = list(range(1, fts, 3))
        z_cds = list(range(2, fts, 3))
        points = np.transpose(np.array([points[:, x_cds], points[:, y_cds], points[:, z_cds]]), (1,2,0))

    size = (frame_w, frame_h)
    result = cv2.VideoWriter(save_file, 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)

    # mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)]
    mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]
    for frame in points:
        skel_image = gen_skeleton(frame, mapping_list, frame_h, frame_w)
        result.write(skel_image)

    result.release()

In [8]:
skel_res, fh, fw = pose_estimator(pose, '../data/videos/K10_V7/cycling/c4.mp4')

OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



In [9]:
skel_res.shape

(90, 33, 3)

In [10]:
res_arr = np.hstack(np.transpose(skel_res, (1,0,2)))
res_arr.shape

(90, 99)

In [11]:
arr1 = res_arr[:50, :]

In [12]:
end_size = 60
arr_df = pd.DataFrame(data=arr1, columns=[str(i) for i in range(arr1.shape[1])])
curr_size = arr1.shape[0]
fill_points = random.sample(range(10, curr_size-10), end_size-curr_size)

df_data = []
for i, row in arr_df.iterrows():
    df_data.append(row.values)
    if i in fill_points:
        df_data.append(pd.Series(dtype='float64').values)

df_new = pd.DataFrame(df_data)
df_new = df_new.interpolate(method='linear', limit_direction='forward', axis=0)

In [13]:
df_new

,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,96,97,98
0,0.388183,0.199434,-0.267344,0.391558,0.187911,-0.254320,0.394370,0.188090,-0.254334,0.397129,...,-0.076726,0.375136,0.646061,0.021960,0.418868,0.719604,-0.192068,0.361955,0.697262,-0.069528
1,0.388186,0.198245,-0.239880,0.391742,0.186632,-0.225882,0.394541,0.186880,-0.225877,0.397248,...,-0.114124,0.375243,0.635843,0.058458,0.418829,0.721542,-0.225323,0.363715,0.694246,-0.012539
2,0.388436,0.198024,-0.247924,0.392426,0.186274,-0.234052,0.395268,0.186582,-0.234050,0.397893,...,-0.099482,0.375993,0.630774,0.067847,0.420380,0.726005,-0.217621,0.364601,0.691327,0.001265
3,0.389098,0.198032,-0.268750,0.393446,0.186287,-0.255215,0.396265,0.186594,-0.255213,0.398761,...,-0.068701,0.376479,0.629760,0.086065,0.422343,0.730273,-0.181047,0.366165,0.689573,0.014418
4,0.390397,0.198053,-0.268658,0.394872,0.186304,-0.255002,0.397538,0.186617,-0.255004,0.399878,...,-0.046252,0.377765,0.629176,0.047736,0.424419,0.737153,-0.154309,0.366202,0.686154,-0.007038
5,0.392104,0.198061,-0.259102,0.396414,0.186331,-0.244143,0.398964,0.186655,-0.244154,0.401252,...,-0.042686,0.377833,0.629171,0.029954,0.426114,0.741278,-0.150714,0.366519,0.683372,-0.038150
6,0.394123,0.198576,-0.235394,0.398113,0.186836,-0.220586,0.400508,0.187165,-0.220602,0.402783,...,-0.055120,0.379566,0.620455,0.017287,0.426547,0.747364,-0.165272,0.366670,0.678409,-0.050862
7,0.395743,0.199227,-0.229779,0.399441,0.187504,-0.215476,0.401713,0.187788,-0.215490,0.404002,...,-0.051387,0.379852,0.608270,0.009776,0.426630,0.751097,-0.153685,0.366915,0.672241,-0.053387
8,0.396185,0.199681,-0.238497,0.399912,0.188047,-0.224562,0.402175,0.188326,-0.224575,0.404480,...,-0.034045,0.380366,0.599976,-0.013454,0.426364,0.755757,-0.139007,0.367393,0.664518,-0.079790
9,0.396409,0.200230,-0.238687,0.400263,0.188632,-0.225236,0.402516,0.188927,-0.225240,0.404843,...,-0.035976,0.381729,0.602513,-0.065703,0.426121,0.760167,-0.144053,0.368950,0.664621,-0.143616


In [14]:
x_cds = list(range(0, 99, 3))
y_cds = list(range(1, 99, 3))
z_cds = list(range(2, 99, 3))
print(len(x_cds), len(z_cds))

33 33


In [15]:
k = df_new.values
xyz_coods = np.transpose(np.array([k[:, x_cds], k[:, y_cds], k[:, z_cds]]), (1,2,0))[:, active_points, :]
xyz_coods.shape

(60, 12, 3)

In [16]:
gen_video(xyz_coods, '../data/pose_data/gen_extended_sub.mp4', fh, fw)

In [17]:
df_new.shape

(60, 99)

In [18]:
df_new

,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,96,97,98
0,0.388183,0.199434,-0.267344,0.391558,0.187911,-0.254320,0.394370,0.188090,-0.254334,0.397129,...,-0.076726,0.375136,0.646061,0.021960,0.418868,0.719604,-0.192068,0.361955,0.697262,-0.069528
1,0.388186,0.198245,-0.239880,0.391742,0.186632,-0.225882,0.394541,0.186880,-0.225877,0.397248,...,-0.114124,0.375243,0.635843,0.058458,0.418829,0.721542,-0.225323,0.363715,0.694246,-0.012539
2,0.388436,0.198024,-0.247924,0.392426,0.186274,-0.234052,0.395268,0.186582,-0.234050,0.397893,...,-0.099482,0.375993,0.630774,0.067847,0.420380,0.726005,-0.217621,0.364601,0.691327,0.001265
3,0.389098,0.198032,-0.268750,0.393446,0.186287,-0.255215,0.396265,0.186594,-0.255213,0.398761,...,-0.068701,0.376479,0.629760,0.086065,0.422343,0.730273,-0.181047,0.366165,0.689573,0.014418
4,0.390397,0.198053,-0.268658,0.394872,0.186304,-0.255002,0.397538,0.186617,-0.255004,0.399878,...,-0.046252,0.377765,0.629176,0.047736,0.424419,0.737153,-0.154309,0.366202,0.686154,-0.007038
5,0.392104,0.198061,-0.259102,0.396414,0.186331,-0.244143,0.398964,0.186655,-0.244154,0.401252,...,-0.042686,0.377833,0.629171,0.029954,0.426114,0.741278,-0.150714,0.366519,0.683372,-0.038150
6,0.394123,0.198576,-0.235394,0.398113,0.186836,-0.220586,0.400508,0.187165,-0.220602,0.402783,...,-0.055120,0.379566,0.620455,0.017287,0.426547,0.747364,-0.165272,0.366670,0.678409,-0.050862
7,0.395743,0.199227,-0.229779,0.399441,0.187504,-0.215476,0.401713,0.187788,-0.215490,0.404002,...,-0.051387,0.379852,0.608270,0.009776,0.426630,0.751097,-0.153685,0.366915,0.672241,-0.053387
8,0.396185,0.199681,-0.238497,0.399912,0.188047,-0.224562,0.402175,0.188326,-0.224575,0.404480,...,-0.034045,0.380366,0.599976,-0.013454,0.426364,0.755757,-0.139007,0.367393,0.664518,-0.079790
9,0.396409,0.200230,-0.238687,0.400263,0.188632,-0.225236,0.402516,0.188927,-0.225240,0.404843,...,-0.035976,0.381729,0.602513,-0.065703,0.426121,0.760167,-0.144053,0.368950,0.664621,-0.143616


In [19]:
def random_padding(arr, output_size):
    # extract and flatten interest locations
    active_locations = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28] 
    sub_arr = arr[:, active_locations, :]
    com_arr = np.hstack(np.transpose(sub_arr, (1,0,2)))

    # pad and interpolate
    arr_df = pd.DataFrame(data=com_arr, columns=[str(i) for i in range(com_arr.shape[1])])
    curr_size = arr_df.shape[0]
    fill_points = random.sample(range(0, curr_size), output_size-curr_size)

    df_data = []
    for i, row in arr_df.iterrows():
        df_data.append(row.values)
        if i in fill_points:
            df_data.append(pd.Series(dtype='float64').values)

    df_new = pd.DataFrame(df_data)
    df_new = df_new.interpolate(method='linear', limit_direction='forward', axis=0)
    return df_new.values 

In [20]:
def random_sampling(arr, output_size, skip_pre=10, skip_post=10):
    # extract and flatten interest locations
    active_locations = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28] 
    sub_arr = arr[:, active_locations, :]
    com_arr = np.hstack(np.transpose(sub_arr, (1,0,2)))

    curr_size = com_arr.shape[0]
    sample_points = random.sample(range(skip_pre, curr_size-skip_post), output_size)
    sample_points.sort()
    out_arr = com_arr[sample_points, :]
    return out_arr

In [21]:
# generated fixed length skeleton seq. 
seq_len = 60 
all_seq = []
all_actions = []
all_ids = []

for f in glob('../data/nipun_video_dataset/PAMAP2_K10_V1/*/*'):
    _, action, p  = f.split('\\')
    file_id = p.split('.')[0]
    skel_res, _, _= pose_estimator(pose, f)
    cur_len = skel_res.shape[0]

    if cur_len < seq_len:
        try:
            res_seq = random_padding(skel_res, seq_len)
        except:
            res_seq = []
    elif (cur_len > seq_len) and (cur_len < seq_len + 25):
        res_seq = random_sampling(skel_res, seq_len, 0, 0)
    elif cur_len > seq_len + 25:
        res_seq = random_sampling(skel_res, seq_len, 10, 10)
    
    if res_seq != []:
        all_seq.append(res_seq)
        all_actions.append(action)
        all_ids.append(file_id)

all_arr = np.array(all_seq )

'NoneType' object has no attribute 'landmark'
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

'NoneType' object has no attribute 'landmark'


C:\Users\ACER\AppData\Local\Temp\ipykernel_21428\924317047.py:23: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if res_seq != []:


OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

'NoneType' object has no attribute 'landmark'
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

'NoneType' object has no attribute 'landmark'
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

'NoneType' object has no attribute 'landmark'
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_

C:\Users\ACER\AppData\Local\Temp\ipykernel_21428\924317047.py:23: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if res_seq != []:


'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType'

...

'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute

C:\Users\ACER\AppData\Local\Temp\ipykernel_21428\924317047.py:23: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if res_seq != []:


OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

'NoneType' object has no attribute 'landmark'
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

'NoneType' object has no attribute 'landmark'
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

'NoneType' object has no attribute 'landmark'
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

'NoneType' object has no attribute 'landmark'
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

'NoneT

C:\Users\ACER\AppData\Local\Temp\ipykernel_21428\924317047.py:23: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if res_seq != []:


OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

'NoneType' object has no attribute 'landmark'
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

'NoneType' object has no attribute 'landmark'
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

'NoneType' object has no attribute 'landmark'
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

OpenCV(4.6.0) D:\a\opencv-python\opencv-python\openc

In [22]:
all_arr.shape

(180, 60, 36)

In [23]:
a,b = np.unique(all_actions, return_counts=True)
dict(zip(a,b))

{'Nordic walking': 10,
 'ascending stairs': 10,
 'car driving': 10,
 'computer work': 10,
 'cycling': 10,
 'descending stairs': 10,
 'folding laundry': 10,
 'house cleaning': 10,
 'ironing': 10,
 'lying': 10,
 'playing soccer': 10,
 'rope jumping': 10,
 'running': 10,
 'sitting': 10,
 'standing': 10,
 'vacuum cleaning': 10,
 'walking': 10,
 'watching TV': 10}

In [24]:
all_ids

['as10',
 'as12',
 'as13',
 'as14',
 'as15',
 'as16',
 'as18',
 'as7',
 'as8',
 'as9',
 'cd11',
 'cd13',
 'cd14',
 'cd16',
 'cd18',
 'cd19',
 'cd2',
 'cd5',
 'cd6',
 'cd9',
 'c10',
 'c11',
 'c13',
 'c17',
 'c18',
 'c19',
 'c2',
 'c23',
 'c24',
 'c6',
 'c10',
 'c14',
 'c16',
 'c17',
 'c18',
 'c2',
 'c20',
 'c4',
 'c6',
 'c9',
 'ds1',
 'ds10',
 'ds12',
 'ds13',
 'ds18',
 'ds20',
 'ds22',
 'ds5',
 'ds7',
 'ds9',
 'fl10',
 'fl12',
 'fl13',
 'fl16',
 'fl18',
 'fl2',
 'fl4',
 'fl5',
 'fl6',
 'fl8',
 'hc11',
 'hc13',
 'hc17',
 'hc18',
 'hc2',
 'hc20',
 'hc22',
 'hc5',
 'hc7',
 'hc9',
 'i12',
 'i13',
 'i16',
 'i19',
 'i21',
 'i26',
 'i28',
 'i32',
 'i4',
 'i7',
 'l10',
 'l11',
 'l12',
 'l13',
 'l16',
 'l17',
 'l18',
 'l20',
 'l3',
 'l4',
 'nw1',
 'nw10',
 'nw14',
 'nw16',
 'nw18',
 'nw20',
 'nw3',
 'nw5',
 'nw6',
 'nw8',
 'ps1',
 'ps10',
 'ps12',
 'ps13',
 'ps18',
 'ps20',
 'ps3',
 'ps5',
 'ps7',
 'ps9',
 'rj1',
 'rj13',
 'rj14',
 'rj15',
 'rj18',
 'rj20',
 'rj3',
 'rj5',
 'rj6',
 'rj9',
 'r1'

In [25]:
full_data = {'actions': all_actions, 'sequences': all_arr, 'id': all_ids}
np.savez('../data/skeleton_k10_v8_movements.npz', all_actions, all_arr, all_ids)

In [27]:
reload_data = np.load('../data/skeleton_k10_v8_movements.npz')
reload_data['arr_1'].shape

(180, 60, 36)

---

In [28]:
import os 
from glob import glob 
import numpy as np 
import cv2

In [29]:
def gen_skeleton(frame, connections, height, width):
    img_3 = np.zeros([height, width,3],dtype=np.uint8)
    img_3.fill(255)

    # add circles 
    for coord in frame:
        x, y = int(width*coord[0]), int(height*coord[1])
        img_3 = cv2.circle(img_3, center=(x,y), radius=1, color=(255, 0, 0), thickness=6)

    # add lines 
    mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]
    for line in mapping_list:
        i, j = line
        st = frame[i, :]
        start_point = (int(width*st[0]), int(height*st[1]))

        en = frame[j, :]
        end_point = (int(width*en[0]), int(height*en[1]))

        img3_ = cv2.line(img_3, start_point, end_point, color=(0, 0, 0), thickness=3)

    return img_3


In [30]:
def gen_video(points, save_file, frame_h, frame_w):
    # make 3D if points are flatten 
    if len(points.shape) == 2:
        fts = points.shape[1]
        x_cds = list(range(0, fts, 3))
        y_cds = list(range(1, fts, 3))
        z_cds = list(range(2, fts, 3))
        points = np.transpose(np.array([points[:, x_cds], points[:, y_cds], points[:, z_cds]]), (1,2,0))

    size = (frame_w, frame_h)
    result = cv2.VideoWriter(save_file, 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)

    # mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)]
    mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]
    for frame in points:
        skel_image = gen_skeleton(frame, mapping_list, frame_h, frame_w)
        result.write(skel_image)

    result.release()

In [31]:
reload_data = np.load('../data/skeleton_k10_movements.npz')
reload_data.files

['arr_0', 'arr_1', 'arr_2']

In [32]:
root_dir = '../data/generated_k10_skels/'
video_h, video_w = 640, 480
action_dict = {}

for i, act in enumerate(reload_data['arr_0']):
    action_dir = os.path.join(root_dir, act)
    if act in action_dict:
        action_dict[act] += 1 
    else:
        action_dict[act] = 0 
        os.makedirs(action_dir,exist_ok=True)

    file_name = reload_data['arr_2'][i]+'.mp4'
    file_dir = os.path.join(action_dir, file_name)
    gen_video(reload_data['arr_1'][i, ...], file_dir, video_h, video_w)
    